# Loading Data

In [1]:
!pip install datasets

In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import tqdm
import gensim
import random
import copy
import unicodedata
from bs4 import BeautifulSoup
import string
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
import nltk


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
from datasets import load_dataset

dataset = load_dataset("emotone_ar")

Generating train split:   0%|          | 0/10065 [00:00<?, ? examples/s]

Dataset emotone_ar downloaded and prepared to /root/.cache/huggingface/datasets/emotone_ar/default/0.0.0/36bfa7e8bdf640ccb3d5701933cfde2fa7051f22881cfab2962122771d32c430. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['tweet', 'label'],
        num_rows: 10065
    })
})

In [5]:
import pandas as pd
Data= pd.DataFrame(dataset['train'])

In [36]:
Data.head()

,tweet,label
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,0
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,1
2,كتنا نيله ف حظنا الهباب xD,3
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,2
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,0


In [6]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10065 entries, 0 to 10064
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   10065 non-null  object
 1   label   10065 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 157.4+ KB


In [11]:
Data['label'].value_counts()

0    1550
1    1444
2    1281
3    1256
4    1220
7    1207
5    1062
6    1045
Name: label, dtype: int64

In [6]:
label_to_class = {
    0: 'none',
    1: 'anger',
    2: 'joy',
    3: 'sadness',
    4: 'love',
    5: 'sympathy',
    6: 'surprise',
    7: 'fear'
}


# EMOJIS

In [6]:
!pip install demoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.5 MB/s eta 0:00:00


In [7]:
import demoji
import re

def extract_emojis(tweet):
  return demoji.findall(tweet)
def extract_emoticons(text):
  emoticon_pattern = r'(?::|;|=)(?:-)?(?:\)|\(|D|P)'
  emoticons = re.findall(emoticon_pattern, text)
  return emoticons

In [8]:
all_emojis = []
all_emoticons = []

for text in Data['tweet']:
    emojis = extract_emojis(text)
    emoticons = extract_emoticons(text)
    all_emojis.extend(emojis)
    all_emoticons.extend(emoticons)

print("All Emojis:", set(all_emojis))
print("All Emoticons:", set(all_emoticons))


All Emojis: {'👸', '🍂', '👻', '🎵', '🚬', '🏃🏻\u200d♀️', '☘', '⚠️', '✌', '💎', '🌷', '🗣', '🙏', '🐓', '💵', '😨', '🇪🇬', '👊🏽', '🇧🇷', '🌸', '🤔', '😌', '👽', '❌', '😐', '🍷', '😆', '🌿', '😖', '🙈', '😞', '😄', '➿', '😑', '❤', '😇', '😚', '💫', '👀', '👏🏽', '👉', '😉', '☹️', '🔑', '👎', '🍁', '🗨', '🔪', '📝', '1️⃣', '❄️', '✋', '💪🏻', '💪', '💜', '😩', '⚽', '💃', '🎼', '🤕', '🎩', '🔫', '😟', '📩', '💃🏼', '🇯🇴', '☔', '👆', '😛', '☁️', '🙂', '💓', '👏🏼', '🙃', '🎀', '🚫', '😊', '🚶🏾\u200d♀️', '‼', '🚶🏽', '👊🏼', '🎷', '😷', '🚶🏻', '😡', '🐤', '😂', '☺️', '😈', '🕋', '👐', '🎶', '🚶', '👎🏼', '😀', '😳', '😍', '⭕', '🤗', '🏴', '✨', '🌟', '🙁', '😅', '💕', '✏', '💟', '🌹', '😿', '😢', '📮', '🙏🏼', '👬', '📍', '🏃🏼', '🇧🇭', '😻', '🐣', '💖', '🐸', '🎉', '😴', '😧', '🎆', '🔴', '◾', '😕', '💚', '😦', '❣', '☄', '😔', '🌼', '🍳', '😫', '☕', '👌🏼', '👃', '✖', '🔥', '👎🏻', '💞', '✍🏻', '©', '🏃', '😋', '👣', '🔸', '🇩🇿', '✏️', '😭', '☺', '🌞', '💪🏽', '😶', '🖋', '👏', '🐰', '✌🏼', '👇', '😃', '✈️', '⚜', '😒', '⁉️', '💔', '🏊', '💗', '🙋', '💩', '✔', '😓', '❤️', '⚪', '✊', '👏🏻', '😲', '🎾', '👼', '🏀', '❣️', '🕊', '👌🏽', '👵🏽', '😜', '😣', '™'

In [10]:
all_emojis= set(all_emojis)
all_emoticons= set(all_emoticons)
print("len Emojis:", len(all_emojis))
print("All Emojis:", all_emojis)
print("len Emoticons:", len(all_emoticons))
print("All Emoticons:", all_emoticons)


len Emojis: 276
All Emojis: {'🙍🏻', '💜', '🎧', '📮', '🎩', '☄', '💭', '⭕', '😍', '♨', '🍔', '🎆', '✊', '🍁', '😌', '😰', '🌾', '👃', '⚜', '🚶🏻', '😓', '✋🏻', '🐓', '🇮🇶', '💋', '🎼', '📩', '👊', '🐑', '🖋', '🎵', '☹️', '💵', '©', '😋', '🎷', '👑', '🙌🏼', '😜', '😟', '👏', '💘', '🙉', '😅', '1️⃣', '❤️', '🌼', '⚽', '☘', '👌🏽', '😡', '👇', '✌🏻', '💖', '☕', '😈', '☺️', '👬', '✔', '💟', '👣', '👎🏼', '✌', '💐', '😷', '🌿', '✍🏻', '🙄', '👊🏽', '🔥', '🌷', '™', '😙', 'ℹ', '📍', '❗', '😎', '🕋', '🤕', '💙', '🌚', '☺', '👍', '🚶🏾\u200d♀️', '👎🏻', '✏️', '➿', '👵🏽', '🐸', '✋', '😉', '‼️', '😒', '🚶', '😥', '🏀', '😯', '❣️', '👉', '♥️', '🎉', '🔴', '😢', '💌', '🏃🏻\u200d♀️', '😆', '👏🏽', '🤔', '2️⃣', '✏', '💪🏽', '🙇', '😩', '👊🏿', '💪', '🇩🇿', '💦', '😄', '🕊', '💪🏼', '😀', '📝', '😤', '😿', '💔', '😔', '😳', '✋🏼', '⚠️', '👧', '♥', '☝', '✖', '👌🏼', '🐣', '🐤', '😻', '🚶🏽', '🤗', '🌟', '🐔', '💞', '🙆🏻', '💚', '🙏🏼', '✌🏼', '🕶', '💝', '💪🏻', '✒', '👍🏻', '🍳', '🎾', '😧', '👍🏼', '🇯🇴', '🇪🇬', '🍃', '♦️', '😑', '😪', '😁', '🎥', '☁️', '😚', '💀', '🏃🏼', '🙋', '😇', '😂', '👼', '👽', '✈️', '🔑', '🐰', '🙌', '😨', '😃', '😶', '🌞', '👌', '👌🏻'

## TTT

In [9]:

updated_emojis = {
    '😩': 'حزن',
    '🚫': 'حزن',
    '♥': 'حب',
    '😔': 'حزن',
    '🇸🇦': 'تعاطف',
    '✌🏼': 'تعاطف',
    '🐸': 'سخريه',
    '😠': 'غضب',
    '😭': 'حزن',
    '🎧': 'لا شيء',
    '™': 'لا شيء',
    '☺️': 'فرح',
    '👋': 'لا شيء',
    '👌🏼': 'لا شيء',
    '💕': 'حب',
    '😓': 'حزن',
    '🙊': 'سخريه',
    '🌚': 'لا شيء',
    '👊🏽': 'غضب',
    '💛': 'حب',
    '💦': 'لا شيء',
    '😤': 'غضب',
    '🍃': 'لا شيء',
    '♨': 'لا شيء',
    '😛': 'فرح',
    '✏️': 'لا شيء',
    '⚪': 'لا شيء',
    '🙁': 'حزن',
    '💪🏼': 'تعاطف',
    '👸': 'لا شيء',
    '☠️': 'لا شيء',
    '🇧🇭': 'فرح',
    '😮': 'مفاجأة',
    '😫': 'حزن',
    '👻': 'مفاجأة',
    '💚': 'حب',
    '🌺': 'حب',
    '🔝': 'لا شيء',
    '✌🏻': 'تعاطف',
    '🙈': 'لا شيء',
    '💗': 'حب',
    '👃': 'لا شيء',
    '😌': 'لا شيء',
    '👐🏽': 'لا شيء',
    '💘': 'حب',
    '😖': 'حزن',
    '🎻': 'لا شيء',
    '©': 'لا شيء',
    '💜': 'حب',
    '👎🏻': 'لا شيء',
    '💓': 'حب',
    '🏊': 'لا شيء',
    '🚶🏽': 'لا شيء',
    '💟': 'حب',
    '👊🏿': 'غضب',
    '👉': 'لا شيء',
    '🌹': 'حب',
    '😁': 'فرح',
    '♦️': 'لا شيء',
    '🏴': 'لا شيء',
    '✔': 'لا شيء',
    '💪🏽': 'تفاؤل',
    '👍🏻': 'لا شيء',
    '😱': 'خوف',
    '🌼': 'حب',
    '✖': 'لا شيء',
    '👀': 'لا شيء',
    '💝': 'حب',
    '🗣': 'لا شيء',
    '🕶': 'لا شيء',
    '😢': 'حزن',
    '❤️': 'حب',
    '🍁': 'حب',
    '💐': 'هدية',
    '👌🏽': 'لا شيء',
    '⚽': 'لا شيء',
    '💡': 'لا شيء',
    '🔕': 'لا شيء',
    '☝': 'لا شيء',
    '🙋': 'غضب',
    '😨': 'خوف',
    '💪': 'لا شيء',
    '✌️': 'تعاطف',
    '👽': 'خوف',
    '🔑': 'لا شيء',
    '✈️': 'لا شيء',
    '😻': 'لا شيء',
    '🎩': 'لا شيء',
    '😷': 'لا شيء',
    '🙏': 'تعاطف',
    'ℹ': 'لا شيء',
    '👎🏼': 'غضب',
    '◾': 'لا شيء',
    '😋': 'فرح',
    '😯': 'مفاجأة',
    '🙂': 'خوف',
    '✍🏻': 'لا شيء',
    '😣': 'حزن',
    '🇯🇴': 'تعاطف',
    '🐣': 'لا شيء',
    '👏🏽': 'فرح',
    '💫': 'لا شيء',
    '🌸': 'لا شيء',
    '😂': 'سخريه',
    '💭': 'حب',
    '😲': 'مفاجأة',
    '🙄': 'لا شيء',
    '💎': 'لا شيء',
    '💙': 'حب',
    '✋🏼': 'لا شيء',
    '🍔': 'لا شيء',
    '🏀': 'لا شيء',
    '🎬': 'لا شيء',
    '😈': 'لا شيء',
    '👊🏼': 'غضب',
    '👣': 'لا شيء',
    '✊': 'لا شيء',
    '😥': 'حزن',
    '⚠️': 'لا شيء',
    '✋🏻': 'لا شيء',
    '😆': 'فرح',
    '😏': 'سخريه',
    '💤': 'لا شيء',
    '📍': 'لا شيء',
    '🙏🏼': 'لا شيء',
    '🤕': 'جرح',
    '🎵': 'لا شيء',
    '💌': 'حب',
    '👐': 'لا شيء',
    '🎉': 'فرح',
    '👏🏻': 'فرح',
    '🇧🇷': 'فرح',
    '😉': 'فرح',
    '🌾': 'لا شيء',
    '😅': 'فرح',
    '😰': 'خوف',
    '😐': 'حزن',
    '🔐': 'لا شيء',
    '☂': 'لا شيء',
    '🚬': 'لا شيء',
    '🙌🏼': 'لا شيء',
    '🎼': 'لا شيء',
    '☄': 'لا شيء',
    '💖': 'حب',
    '🐔': 'لا شيء',
    '💔': 'حزن',
    '⚜': 'لا شيء',
    '1️⃣': 'لا شيء',
    '😑': 'لا شيء',
    '✋': 'رأي',
    '💉': 'لا شيء',
    '😳': 'مفاجأة',
    '✌': 'تعاطف',
    '👍🏼': 'لا شيء',
    '2️⃣': 'لا شيء',
    '👌🏻': 'لا شيء',
    '👏🏼': 'فرح',
    '💀': 'لا شيء',
    '😬': 'لا شيء',
    '😹': 'لا شيء',
    '🕋': 'لا شيء',
    '👑': 'لا شيء',
    '📮': 'لا شيء',
    '🚶': 'لا شيء',
    '🇮🇶': 'تعاطف',
    '😒': 'لا شيء',
    '👎': 'حزن',
    '🎈': 'لا شيء',
    '🍷': 'لا شيء',
    '🌷': 'حب',
    '🔪': 'لا شيء',
    '😟': 'حزن',
    '🎀': 'لا شيء',
    '🎾': 'لا شيء',
    '👯': 'لا شيء',
    '‼': 'لا شيء',
    '😴': 'حب',
    '🔴': 'لا شيء',
    '😪': 'حزن',
    '👬': 'لا شيء',
    '💵': 'لا شيء',
    '🖋': 'لا شيء',
    '📩': 'لا شيء',
    '☺': 'فرح',
    '❤': 'حب',
    '📝': 'لا شيء',
    '👇': 'لا شيء',
    '😧': 'مفاجأة',
    '‼️': 'لا شيء',
    '⭕': 'لا شيء',
    '😚': 'لا شيء',
    '😡': 'غضب',
    '🙍🏻': 'حزن',
    '👵🏽': 'لا شيء',
    '😎': 'لا شيء',
    '🏃': 'لا شيء',
    '❗': 'لا شيء',
    '🍂': 'لا شيء',
    '🤗': 'سخريه',
    '🎆': 'لا شيء',
    '😜': 'فرح',
    '⁉️': 'مفاجأة',
    '💃': 'لا شيء',
    '❣️': 'لا شيء',
    '☁️': 'لا شيء',
    '👈': 'لا شيء',
    '😞': 'حزن',
    '🌿': 'حب',
    '✒': 'لا شيء',
    '❣': 'لا شيء',
    '💞': 'حب',
    '🍟': 'لا شيء',
    '😘': 'حب',
    '😦': 'لا شيء',
    '🙆🏻': 'خوف',
    '🇩🇿': 'تعاطف',
    '👍': 'لا شيء',
    '💂🏿': 'لا شيء',
    '😄': 'فرح',
    '💪🏻': 'لا شيء',
    '💃🏼': 'لا شيء',
    '😃': 'فرح',
    '☹️': 'حزن',
    '🚶🏻': 'لا شيء',
    '🌟': 'فرح',
    '💋': 'حب',
    '❌': 'لا شيء',
    '🙌': 'لا شيء',
    '🕊': 'لا شيء',
    '🔸': 'لا شيء',
    '🇪🇬': 'لا شيء',
    '🙇': 'حزن',
    '❄️': 'لا شيء',
    '🐑': 'لا شيء',
    '👌': 'لا شيء',
    '😙': 'لا شيء',
    '☔': 'لا شيء',
    '🏃🏼': 'لا شيء',
    '🎷': 'لا شيء',
    '🚶🏾\u200d♀️': 'لا شيء',
    '✏': 'لا شيء',
    '😊': 'فرح',
    '🌝': 'لا شيء',
    '☕': 'لا شيء',
    '💩': 'لا شيء',
    '🎶': 'حب',
    '🐰': 'لا شيء',
    '🐓': 'لا شيء',
    '😿': 'خوف',
    '👆': 'لا شيء',
    '😍': 'حب',
    '🙉': 'سخريه',
    '😀': 'فرح',
    '👏': 'لا شيء',
    '🙏🏻': 'تعاطف',
    '🔫': 'لا شيء',
    '🙃': 'لا شيء',
    '👼': 'لا شيء',
    '🎤': 'لا شيء',
    '😇': 'لا شيء',
    '🍳': 'لا شيء',
    '🐤': 'لا شيء',
    '😶': 'خوف',
    '🌞': 'لا شيء',
    '👊': 'غضب',
    '🔥': 'لا شيء',
    '🤔': 'لا شيء',
    '👧': 'لا شيء',
    '➿': 'لا شيء',
    '🗨': 'لا شيء',
    '🎥': 'لا شيء',
    '☘': 'لا شيء',
    '♥️': 'حب',
    '✨': 'لا شيء',
    '😕': 'حزن',
    '🏃🏻\u200d♀️': 'لا شيء'
}




In [10]:
filtered_emojis = {
    '😩': 'حزن',
    '🚫': 'حزن',
    '♥': 'حب',
    '😔': 'حزن',
    '🇸🇦': 'تعاطف',
    '✌🏼': 'تعاطف',
    '🐸': 'سخريه',
    '😠': 'غضب',
    '😭': 'حزن',
    '🎧': '',
    '™': '',
    '☺️': 'فرح',
    '👋': '',
    '👌🏼': '',
    '💕': 'حب',
    '😓': 'حزن',
    '🙊': 'سخريه',
    '🌚': '',
    '👊🏽': 'غضب',
    '💛': 'حب',
    '💦': '',
    '😤': 'غضب',
    '🍃': '',
    '♨': '',
    '😛': 'فرح',
    '✏️': '',
    '⚪': '',
    '🙁': 'حزن',
    '💪🏼': 'تعاطف',
    '👸': '',
    '☠️': '',
    '🇧🇭': 'فرح',
    '😮': 'مفاجأة',
    '😫': 'حزن',
    '👻': 'مفاجأة',
    '💚': 'حب',
    '🌺': 'حب',
    '🔝': '',
    '✌🏻': 'تعاطف',
    '🙈': '',
    '💗': 'حب',
    '👃': '',
    '😌': '',
    '👐🏽': '',
    '💘': 'حب',
    '😖': 'حزن',
    '🎻': '',
    '©': '',
    '💜': 'حب',
    '👎🏻': '',
    '💓': 'حب',
    '🏊': '',
    '🚶🏽': '',
    '💟': 'حب',
    '👊🏿': 'غضب',
    '👉': '',
    '🌹': 'حب',
    '😁': 'فرح',
    '♦️': '',
    '🏴': '',
    '✔': '',
    '💪🏽': 'تفاؤل',
    '👍🏻': '',
    '😱': 'خوف',
    '🌼': 'حب',
    '✖': '',
    '👀': '',
    '💝': 'حب',
    '🗣': '',
    '🕶': '',
    '😢': 'حزن',
    '❤️': 'حب',
    '🍁': 'حب',
    '💐': 'هدية',
    '👌🏽': '',
    '⚽': '',
    '💡': '',
    '🔕': '',
    '☝': '',
    '🙋': 'غضب',
    '😨': 'خوف',
    '💪': '',
    '✌️': 'تعاطف',
    '👽': 'خوف',
    '🔑': '',
    '✈️': '',
    '😻': '',
    '🎩': '',
    '😷': '',
    '🙏': 'تعاطف',
    'ℹ': '',
    '👎🏼': 'غضب',
    '◾': '',
    '😋': 'فرح',
    '😯': 'مفاجأة',
    '🙂': 'خوف',
    '✍🏻': '',
    '😣': 'حزن',
    '🇯🇴': 'تعاطف',
    '🐣': '',
    '👏🏽': 'فرح',
    '💫': '',
    '🌸': '',
    '😂': 'سخريه',
    '💭': 'حب',
    '😲': 'مفاجأة',
    '🙄': '',
    '💎': '',
    '💙': 'حب',
    '✋🏼': '',
    '🍔': '',
    '🏀': '',
    '🎬': '',
    '😈': '',
    '👊🏼': 'غضب',
    '👣': '',
    '✊': '',
    '😥': 'حزن',
    '⚠️': '',
    '✋🏻': '',
    '😆': 'فرح',
    '😏': 'سخريه',
    '💤': '',
    '📍': '',
    '🙏🏼': '',
    '🤕': 'جرح',
    '🎵': '',
    '💌': 'حب',
    '👐': '',
    '🎉': 'فرح',
    '👏🏻': 'فرح',
    '🇧🇷': 'فرح',
    '😉': 'فرح',
    '🌾': '',
    '😅': 'فرح',
    '😰': 'خوف',
    '😐': 'حزن',
    '🔐': '',
    '☂': '',
    '🚬': '',
    '🙌🏼': '',
    '🎼': '',
    '☄': '',
    '💖': 'حب',
    '🐔': '',
    '💔': 'حزن',
    '⚜': '',
    '1️⃣': '',
    '😑': '',
    '✋': 'رأي',
    '💉': '',
    '😳': 'مفاجأة',
    '✌': 'تعاطف',
    '👍🏼': '',
    '2️⃣': '',
    '👌🏻': '',
    '👏🏼': 'فرح',
    '💀': '',
    '😬': '',
    '😹': '',
    '🕋': 'حب',
    '👑': '',
    '📮': '',
    '🚶': '',
    '🇮🇶': 'تعاطف',
    '😒': '',
    '👎': 'حزن',
    '🎈': '',
    '🍷': '',
    '🌷': 'حب',
    '🔪': '',
    '😟': 'حزن',
    '🎀': '',
    '🎾': '',
    '👯': '',
    '‼': '',
    '😴': 'حب',
    '🔴': '',
    '😪': 'حزن',
    '👬': '',
    '💵': '',
    '🖋': '',
    '📩': '',
    '☺': 'فرح',
    '❤': 'حب',
    '📝': '',
    '👇': '',
    '😧': 'مفاجأة',
    '‼️': '',
    '⭕': '',
    '😚': '',
    '😡': 'غضب',
    '🙍🏻': 'حزن',
    '👵🏽': '',
    '😎': '',
    '🏃': '',
    '❗': '',
    '🍂': '',
    '🤗': 'سخريه',
    '🎆': '',
    '😜': 'فرح',
    '⁉️': 'مفاجأة',
    '💃': '',
    '❣️': '',
    '☁️': '',
    '👈': '',
    '😞': 'حزن',
    '🌿': 'حب',
    '✒': '',
    '❣': '',
    '💞': 'حب',
    '🍟': '',
    '😘': 'حب',
    '😦': '',
    '🙆🏻': 'خوف',
    '🇩🇿': 'تعاطف',
    '👍': '',
    '💂🏿': '',
    '😄': 'فرح',
    '💪🏻': '',
    '💃🏼': '',
    '😃': 'فرح',
    '☹️': 'حزن',
    '🚶🏻': '',
    '🌟': 'فرح',
    '💋': 'حب',
    '❌': '',
    '🙌': '',
    '🕊': '',
    '🔸': '',
    '🇪🇬': '',
    '🙇': 'حزن',
    '❄️': '',
    '🐑': '',
    '👌': '',
    '😙': '',
    '☔': '',
    '🏃🏼': '',
    '🎷': '',
    '🚶🏾\u200d♀️': '',
    '✏': '',
    '😊': 'فرح',
    '🌝': '',
    '☕': '',
    '💩': '',
    '🎶': 'حب',
    '🐰': '',
    '🐓': '',
    '😿': 'خوف',
    '👆': '',
    '😍': 'حب',
    '🙉': 'سخريه',
    '😀': 'فرح',
    '👏': '',
    '🙏🏻': 'تعاطف',
    '🔫': '',
    '🙃': '',
    '👼': '',
    '🎤': '',
    '😇': '',
    '🍳': '',
    '🐤': '',
    '😶': 'خوف',
    '🌞': '',
    '👊': 'غضب',
    '🔥': '',
    '🤔': '',
    '👧': '',
    '➿': '',
    '🗨': '',
    '🎥': '',
    '☘': '',
    '♥️': 'حب',
    '✨': '',
    '😕': 'حزن',
    '🏃🏻\u200d♀️': ''
}







In [11]:
emoticons_to_labels = {
    ':)': 'فرح',
    ':(': 'حزن',
    '=)': 'فرح',
    ';)': 'مفاجأة',
    ':D': 'فرح',
    '=D': 'فرح',
    'xD':'فرح'
}

In [17]:
# Assume emoji_to_label is a dictionary mapping emojis to classes
# and tweets_data is a list of your tweets

# Step 1: Create a dictionary to store emojis and their corresponding tweets
emojis_with_tweets = {}

# Step 2: Loop through tweets
for idx, tweet in enumerate(Data['tweet']):
    emojis_in_tweet = extract_emojis(tweet)  # Extract emojis from tweet
    emojis_classes = [updated_emojis.get(emoji, 'لا شيء') for emoji in emojis_in_tweet]
    
    # Step 3: Check if emojis correspond to the "none" class
    if 'لا شيء' in emojis_classes:
        for emoji in emojis_in_tweet:
            if emoji in emojis_with_tweets:
                emojis_with_tweets[emoji].append(tweet)
            else:
                emojis_with_tweets[emoji] = [tweet]

# Step 4: Print emojis and their corresponding tweets
for emoji, tweet_list in emojis_with_tweets.items():
    print(f"Emoji: {emoji}")
    for tweet in tweet_list:
        print(f"Tweet: {tweet}")
        print()


Emoji: 😏
Tweet: باي موراي ابن الجزمه ده ياخد ذهبيه السنه دي وذهبيه الاوليمبياد اللي فاتت 😏

Tweet: بيتريقوا ان لاعبات محجبات بيشاركوا في الاوليمبياد ..علي اساس ان 80%من المصريات مش محجبات..ولا علي اساس اننا في الستينات مثلا.😏

Tweet: الدول كلها رايحه تجيب ميداليات دهب ف الاوليمبياد ومصر اكبر بعثه تقريبا ف البطوله مع الصين ومش عارفين نجيب الا ميداليتين برونز    حسبي الله 😏

Tweet: لا مؤاخذه يا مخ هو المنتخب وصل الاوليمبياد و انا معرفش ؟ 😏😂

Tweet: شوفي كده تلات اربع اللعيبه اللي طالعين الاوليمبياد...فشله بمعني الكلمه..في ابطال كتير احسن منهم هنا برده بس ده ابن فلان😏

Tweet: ستانلي موقوف 😏😏وتقريبا اجاي فس الاوليمبياد

Tweet: مراه ترقص امام الرجال يعني الحييييين عادي نروح الاوليمبياد و نقعد نتنطنط و نتقلب ... لكن هذه اللي بالكاد يسمي رقص حرام و عيب ؟ 😏

Tweet: بفضل قاعده اتفرج ع الاوليمبياد واتحسر علي حياتي واحس اني مليش لازمه ف العالم 😏

Tweet: هو انا لوحدي اللي مش عاجبني فستان ال ushers في الاوليمبياد؟! 😏

Tweet: بيتكلموا علي ساره سمير الي شابت الثانويه وراحت الاوليمبياد وازاي تسيب التع

In [ ]:

# Step 1: Create a dictionary to store emojis and their corresponding tweets
emojis_with_tweets = {}

# Step 2: Loop through tweets in the dataset
for idx, row in Data.iterrows():
    tweet = row['tweet']
    label = row['label']
    emojis_in_tweet = extract_emojis(tweet)  # Extract emojis from tweet
    emojis_classes = [updated_emojis.get(emoji, 'لا شيء') for emoji in emojis_in_tweet]
    
    # Step 3: Check if emojis correspond to the "none" class
    if 'لا شيء' in emojis_classes:
        for emoji in emojis_in_tweet:
            if emoji in emojis_with_tweets:
                emojis_with_tweets[emoji].append((tweet, label))
            else:
                emojis_with_tweets[emoji] = [(tweet, label)]

# Step 4: Print emojis, tweets, and their corresponding labels
for emoji in sorted(emojis_with_tweets.keys()):
    tweet_list = emojis_with_tweets[emoji]
    print(f"Emoji: {emoji}")
    for tweet, label in tweet_list:
        print(f"Tweet: {tweet}")
        print(f"Label: {label}")
        print()


In [19]:
emoji_to_label = {**updated_emojis, **emoticons_to_labels}

filtered_labels = {emoji: label for emoji, label in emoji_to_label.items() if label != 'لا شيء'}

print(len(filtered_labels))
print(filtered_labels)

121
{'😩': 'حزن', '🚫': 'حزن', '♥': 'حب', '😔': 'حزن', '✌🏼': 'تعاطف', '🐸': 'سخريه', '😠': 'غضب', '😭': 'حزن', '☺️': 'فرح', '💕': 'حب', '😓': 'حزن', '🙊': 'سخريه', '👊🏽': 'غضب', '💛': 'حب', '😤': 'غضب', '😛': 'فرح', '🙁': 'حزن', '💪🏼': 'تعاطف', '🇧🇭': 'فرح', '😮': 'مفاجأة', '😫': 'حزن', '👻': 'مفاجأة', '💚': 'حب', '🌺': 'حب', '✌🏻': 'تعاطف', '💗': 'حب', '💘': 'حب', '😖': 'حزن', '💜': 'حب', '💓': 'حب', '💟': 'حب', '👊🏿': 'غضب', '🌹': 'حب', '😁': 'فرح', '💪🏽': 'تفاؤل', '😱': 'خوف', '🌼': 'حب', '💝': 'حب', '😢': 'حزن', '❤️': 'حب', '🍁': 'حب', '💐': 'هدية', '🙋': 'غضب', '😨': 'خوف', '✌️': 'تعاطف', '👽': 'خوف', '🙏': 'تعاطف', '👎🏼': 'غضب', '😋': 'فرح', '😯': 'مفاجأة', '🙂': 'خوف', '😣': 'حزن', '🇯🇴': 'تعاطف', '👏🏽': 'فرح', '😂': 'سخريه', '💭': 'حب', '😲': 'مفاجأة', '💙': 'حب', '👊🏼': 'غضب', '😥': 'حزن', '😆': 'فرح', '😏': 'سخريه', '🤕': 'جرح', '💌': 'حب', '🎉': 'فرح', '👏🏻': 'فرح', '🇧🇷': 'فرح', '😉': 'فرح', '😅': 'فرح', '😰': 'خوف', '😐': 'حزن', '💖': 'حب', '💔': 'حزن', '✋': 'رأي', '😳': 'مفاجأة', '✌': 'تعاطف', '👏🏼': 'فرح', '🇮🇶': 'تعاطف', '👎': 'حزن', '🌷': '

In [12]:
def replace_emojis_with_emotions(tweet, emoji_to_emotion):
    # Loop through each emoji in the dictionary and replace it with its emotion
    for emoji, emotion in emoji_to_emotion.items():
        tweet = tweet.replace(emoji, ' '+ emotion+' ')
    return tweet

In [13]:
emoji_to_label = {**filtered_emojis, **emoticons_to_labels}

Data['tweet_with_out_emojis'] = [replace_emojis_with_emotions(text, emoji_to_label) for text in Data['tweet'] ]


In [24]:
Data.head()


,tweet,label,tweet_with_out_emojis
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,0,الاوليمبياد الجايه هكون لسه ف الكليه ..
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...
2,كتنا نيله ف حظنا الهباب xD,3,كتنا نيله ف حظنا الهباب فرح
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,2,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,0,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...


# Cleaning

In [14]:
%pip install arabic-stopwords
import arabicstopwords.arabicstopwords as ast


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 7.4 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [15]:
stopwords_list = ast.stopwords_list()

negation_prohibition_words = ["ليس", "لا", "لم", "لن"]
stopwords_list = [word for word in stopwords_list if word not in negation_prohibition_words]
stopwords_list.extend(["دي","اﻻ", "ده", "انا","الى","انت","اللى", "ان","ما","يا","يلا","دا","مين","دول","دم","حد","من","عن","على","علشان","أني","وأنا","الل","كده","يعني","احنا","ايه"])


In [72]:
stopwords_list

['أن',
 'أنك',
 'أنكما',
 'أنكم',
 'أنكن',
 'أننا',
 'أنها',
 'أنه',
 'أنهما',
 'أنهم',
 'أنهن',
 'أني',
 'بأن',
 'بأنك',
 'بأنكما',
 'بأنكم',
 'بأنكن',
 'بأننا',
 'بأنها',
 'بأنه',
 'بأنهما',
 'بأنهم',
 'بأنهن',
 'بأني',
 'فأن',
 'فأنك',
 'فأنكما',
 'فأنكم',
 'فأنكن',
 'فأننا',
 'فأنها',
 'فأنه',
 'فأنهما',
 'فأنهم',
 'فأنهن',
 'فأني',
 'فبأن',
 'فبأنك',
 'فبأنكما',
 'فبأنكم',
 'فبأنكن',
 'فبأننا',
 'فبأنها',
 'فبأنه',
 'فبأنهما',
 'فبأنهم',
 'فبأنهن',
 'فبأني',
 'فكأن',
 'فكأنك',
 'فكأنكما',
 'فكأنكم',
 'فكأنكن',
 'فكأننا',
 'فكأنها',
 'فكأنه',
 'فكأنهما',
 'فكأنهم',
 'فكأنهن',
 'فكأني',
 'فلأن',
 'فلأنك',
 'فلأنكما',
 'فلأنكم',
 'فلأنكن',
 'فلأننا',
 'فلأنها',
 'فلأنه',
 'فلأنهما',
 'فلأنهم',
 'فلأنهن',
 'فلأني',
 'كأن',
 'كأنك',
 'كأنكما',
 'كأنكم',
 'كأنكن',
 'كأننا',
 'كأنها',
 'كأنه',
 'كأنهما',
 'كأنهم',
 'كأنهن',
 'كأني',
 'لأن',
 'لأنك',
 'لأنكما',
 'لأنكم',
 'لأنكن',
 'لأننا',
 'لأنها',
 'لأنه',
 'لأنهما',
 'لأنهم',
 'لأنهن',
 'لأني',
 'وأن',
 'وأنك',
 'وأنكما',
 'وأنكم',
 '

In [40]:
'''
function used to remove HTML tags
and the content between the tags
for data preprocessing
'''
import tqdm

def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  # remove iframe & script tags
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  # replace new lines and carriage return with new line
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text


'''
function used to handle accented char
in multilingual scenarios
'''
def remove_accented_chars(text):
  # normalize the text to remove accented char
  text = unicodedata.normalize('NFKD', text)
  # encode it to ASCII
  text = text.encode('ascii', 'ignore')
  # decode it back ti unicode
  text = text.decode('utf-8', 'ignore')
  return text

def remove_punctuations(text):
    punctuations = '''`÷×؛<>_():*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation + string.ascii_letters
    translator = str.maketrans('.', ' ', punctuations)
    return text.translate(translator)


def clean_str(text):
  search = ["أ", "إ", "آ", "ة", "_", "-", "/", ".", "،", " و ", " يا ", '"', "ـ", "'", "ى", "\n", "\t", "?", "؟", "!", "“"]
  replace = ["ا", "ا", "ا", "ه", " ", " ", " ", " ", "،", " و", " يا", "", "", "", "ي", " ", " ", " ?", " ؟", " !", ""]

  #remove tashkeel
  p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
  text = re.sub(p_tashkeel,"", text)

  #remove longation
  p_longation = re.compile(r'(.)\1+')
  subst = r"\1\1"
  text = re.sub(p_longation, subst, text)

  text = text.replace('وو', 'و')
  text = text.replace('يي', 'ي')
  text = text.replace('اا', 'ا')

  for i in range(0, len(search)):
      text = text.replace(search[i], replace[i])

  #trim
  text = text.strip()

  return text




def remove_stop_words(text):
    return ' '.join(word for word in str(text).split() if word not in stopwords_list)

'''
main function for text preprocessing
'''
def pre_process_corpus(docs):
    norm_docs = []

    for doc_index, doc in tqdm.tqdm(enumerate(docs)):
        # Store the original text of the current index
        original_text = doc

        doc = strip_html_tags(doc)
        
        #doc = remove_stop_words(doc)
        
       
        doc = re.sub(' +', ' ', doc)
        doc = doc.strip()
        doc = re.sub(r'\d+', '', doc)
        doc = clean_str(doc)
       
        doc = remove_punctuations(doc)
          
        norm_docs.append(doc)
    
    return norm_docs

In [41]:
Data['cleaned_data']= pre_process_corpus(Data['tweet_with_out_emojis'])

0it [00:00, ?it/s]/tmp/ipykernel_28/4174289219.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")
10065it [00:01, 7049.26it/s]


# Stemming

In [20]:
from nltk.stem import ISRIStemmer
isri_stemmer = ISRIStemmer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
#   Applies stemming to the input Arabic text using the ISRI Arabic stemmer.

def light_stemming(text):
    s = " "
    x = w_tokenizer.tokenize(text)
    return s.join([isri_stemmer.suf32(isri_stemmer.norm(isri_stemmer.waw(w), num=3)) for w in x])


def root_stemming(text):
    s = " "
    x = w_tokenizer.tokenize(text)
    return s.join([isri_stemmer.stem(w) for w in x])

## Light Stemming

In [21]:
Data["Light Stemming"] = Data['cleaned_data'].apply(light_stemming)

## Root Stemming

In [22]:
Data["Root Stemming"] = Data['cleaned_data'].apply(root_stemming)

In [33]:
Data

,tweet,label,tweet_with_out_emojis,cleaned_data,Light Stemming,Root Stemming
0,الاوليمبياد الجايه هكون لسه ف الكليه ..,0,الاوليمبياد الجايه هكون لسه ف الكليه ..,الاوليمبياد الجايه هكون لسه الكليه,الاوليمبياد الجايه هكون لسه الكليه,اوليمبياد جيه هكن لسه كله
1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,1,عجز الموازنه وصل ل93.7 % من الناتج المحلي يعني...,عجز الموازنه وصل ل الناتج المحلي لسه اقل ن...,عجز الموازنه وصل ل الناتج المحلي لسه اقل نفلس ...,عجز وزن وصل ل نتج حلي لسه اقل فلس هيم لسه تبع ...
2,كتنا نيله ف حظنا الهباب xD,3,كتنا نيله ف حظنا الهباب فرح,كتنا نيله حظنا الهباب فرح,كتنا نيله حظنا الهباب فرح,كتن نيل حظن هبب فرح
3,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,2,جميعنا نريد تحقيق اهدافنا لكن تونس تالقت في حر...,نريد تحقيق اهدافنا تونس تالقت حراسه المرمي,نريد تحقيق اهداف تونس تالقت حراسه المرمي,نرد حقق هدف ونس تلق حرس رمي
4,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,0,الاوليمبياد نظامها مختلف .. ومواعيد المونديال ...,الاوليمبياد نظامها مختلف ومواعيد المونديال ...,الاوليمبياد نظام مختلف ومواعيد المونديال مكانت...,اوليمبياد نظم خلف مواعيد مونديال مكانتش قرف حج...
...,...,...,...,...,...,...
10060,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,3,2222: يلا يا جماعه حفله عمرو دياب خلصت نريح شو...,جماعه حفله عمرو دياب خلصت نريح شويه ونبدا نكت...,جماعه حفله عمرو دياب خلصت نريح شويه ونبدا نكتئ...,جمع حفل عمرو ديب خلص نرح شوه نبد كتئب عشم انن ...
10061,Mohamed5: اييييه دااا 😲😲 اوزيييل❤,6,Mohamed5: اييييه دااا مفاجأة مفاجأة اوزيييل...,ايه دا مفاجاه مفاجاه اوزيل حب,ايه دا مفاجاه مفاجاه اوزيل حب,ايه دا فجا فجا وزل حب
10062,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد 😃,0,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد ...,عملتلها ريتويت بمناسبه ساره بتاعه الاوليمبياد فرح,عملتل ريتويت بمناسبه ساره بتاعه الاوليمبياد فرح,عملتل ريتو نسب سره بتع اوليمبياد فرح
10063,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,2,وعليك قبلنا يانجم النجوم ياعندليب الحب والاحساس,قبلنا يانجم النجوم ياعندليب الحب والاحساس,قبل يانجم النجوم ياعندليب الحب والاحساس,قبل نجم نجم ياعندليب لحب حسس


# Split data

In [42]:
df_train, df_test = train_test_split(Data, test_size=0.3, random_state=42)
df_test, df_val = train_test_split(df_test, test_size=0.5, random_state=42)


## Train and test data of light stemmer

---



In [43]:
ls_x_train, ls_y_train = df_train["Light Stemming"], df_train["label"]
ls_x_test, ls_y_test = df_test["Light Stemming"], df_test["label"]


## Train and test data of light stemmer
---


In [30]:
rs_x_train, rs_y_train = df_train["Root Stemming"], df_train["label"]
rs_x_test, rs_y_test = df_test["Root Stemming"], df_test["label"]


# Tockenization && Emebdding

In [24]:
!pip install transformers

In [25]:
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tqdm import tqdm

marbert_model_path = 'UBC-NLP/MARBERT'
tokenizer = AutoTokenizer.from_pretrained(marbert_model_path, from_tf=True)
marbert_model = TFAutoModel.from_pretrained(marbert_model_path, output_hidden_states=True)

All model checkpoint layers were used when initializing TFBertModel.

All the layers of TFBertModel were initialized from the model checkpoint at UBC-NLP/MARBERT.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [18]:
import numpy as np
from tqdm import tqdm
def get_embeddings(token_ids):
    # Convert token IDs to input tensors
    input_ids = tf.convert_to_tensor(token_ids)

    # Get embeddings
    hidden_states = marbert_model(input_ids=input_ids)[2]
    sentence_embd = tf.reduce_mean(tf.reduce_sum(tf.stack(hidden_states[-4:]), axis=0), axis=1)
    return sentence_embd
def get_padded_embeddings(padded_sequences):
    embeddings_list = []

    for seq in tqdm(padded_sequences, desc="Getting embeddings"):
        # Remove padding values (0) from the sequence
       # non_zero_tokens = [token for token in seq if token != 0]
        #if non_zero_tokens:
        embeddings = get_embeddings([seq])
        embeddings_list.append(embeddings)

    if embeddings_list:
        return np.concatenate(embeddings_list, axis=0)
    else:
        return None

In [47]:
import numpy as np
from tqdm import tqdm

def bert_tokenize(texts: str) -> list:
    max_len = 0
    for text in texts:
        max_len = max(len(tokenizer.basic_tokenizer.tokenize(text)), max_len)
    tokens = tokenizer(texts, padding='max_length', truncation=True, max_length=max_len)
    return (tokens['input_ids'], tokens['attention_mask'], tokens['token_type_ids'])
def get_embeddings(token_ids):
    # Convert token IDs to input tensors
    input_ids = tf.convert_to_tensor(token_ids)

    # Get embeddings
    hidden_states = marbert_model(input_ids=input_ids)[2]
    sentence_embd = tf.reduce_mean(tf.reduce_sum(tf.stack(hidden_states[-4:]), axis=0), axis=1)
    return sentence_embd
def get_padded_embeddings(padded_sequences):
    embeddings_list = []

    for seq in tqdm(padded_sequences, desc="Getting embeddings"):
        # Remove padding values (0) from the sequence
        non_zero_tokens = [token for token in seq if token != 0]
        if non_zero_tokens:
            embeddings = get_embeddings([non_zero_tokens])
            embeddings_list.append(embeddings)

    if embeddings_list:
        return np.concatenate(embeddings_list, axis=0)
    else:
        return None

In [26]:
remove_special_tokens=0  #change this to 0 if you want to keep the special token
stemtype = 'Light Stemming'


def bert_tokenize(text: str) -> dict:
  tokens = tokenizer(text, padding='max_length', truncation=True, max_length=50)
  if remove_special_tokens == 1:
    shape = np.array(tokens['input_ids']).shape
    modified_input_ids = np.zeros(shape).astype(np.int32)
    modified_attention_mask = np.zeros(shape).astype(np.int32)
    # Modify the input IDs and attention mask as per your requirement
    for i in range(shape[0]):
      modified_input_ids[i] = [0 if token_id == 1 else 0 if token_id == 3 else 0 if token_id == 0 else 0 if token_id == 2 else 0 if token_id == 4 else token_id for token_id in tokens['input_ids'][i]]
      modified_attention_mask[i] = [0 if token_id in [1, 3, 0, 2, 4] else 1 for token_id in tokens['input_ids'][i]]
    # Update the input IDs and attention mask in the tokens dictionary
    tokens['input_ids'] = modified_input_ids
    tokens['attention_mask'] = modified_attention_mask
  return tokens


def get_embeddings(ids, mask, type_ids):
  ids = tf.convert_to_tensor(ids)
  #print(ids.shape)
  mask = tf.convert_to_tensor(mask)
  #print(mask.shape)
  #print(mask)
  type_ids = tf.convert_to_tensor(type_ids)
  #print(type_ids.shape)
  hidden_states = marbert_model(input_ids=ids, attention_mask=mask, token_type_ids=type_ids)[0]
  averaged_embedding = tf.reduce_mean(hidden_states, axis=1)
  return hidden_states.numpy()

def embedd(text):
  text=text.values.astype(str).tolist()
  tokens = bert_tokenize(text)
  xlen = np.array(tokens['input_ids']).shape[0]
  x_emb = np.zeros((xlen,50,768))
  for i in range(0,xlen,100):
    if(i+100 < xlen):
        input_ids = tokens['input_ids'][i:i+100]
        attention_mask = tokens['attention_mask'][i:i+100]
        token_type_ids = tokens['token_type_ids'][i:i+100]
        x_emb[i:i+100] = get_embeddings(input_ids,attention_mask,token_type_ids)
    else:
        input_ids = tokens['input_ids'][i:xlen]
        attention_mask = tokens['attention_mask'][i:xlen]
        token_type_ids = tokens['token_type_ids'][i:xlen]
        x_emb[i:xlen] = get_embeddings(input_ids,attention_mask,token_type_ids)
  return x_emb


## Emebdding of light stemmer

In [64]:
ls_train_embeddings=embedd(df_train["tweet_with_out_emojis"])
ls_val_embeddings=embedd(df_val["tweet_with_out_emojis"])
ls_test_embeddings =embedd(df_test["tweet_with_out_emojis"])


## Emebdding of root stemmer

In [5]:
import pickle


In [25]:

with open('/kaggle/working/rs_train_embeddings.pkl', 'rb') as f:
    drs_train_embeddings = pickle.load(f)

with open('/kaggle/working/rs_test_embeddings.pkl', 'rb') as f:
    drs_test_embeddings = pickle.load(f)


In [6]:
with open('/kaggle/working/train.pkl', 'rb') as f:
    df_train = pickle.load(f)

with open('/kaggle/working/test.pkl', 'rb') as f:
    df_test = pickle.load(f)


In [19]:
with open('/kaggle/working/ls_train_embeddings.pkl', 'rb') as f:
    ls_train_embeddings = pickle.load(f)

with open('/kaggle/working/ls_test_embeddings.pkl', 'rb') as f:
    ls_test_embeddings = pickle.load(f)

In [40]:
ls_train_embeddings.shape,ls_test_embeddings.shape

((7045, 50, 768), (1510, 50, 768))

In [65]:
test_whole=np.mean(ls_test_embeddings, axis=1)
train_whole=np.mean(ls_train_embeddings, axis=1)
val_whole=np.mean(ls_val_embeddings, axis=1)

In [46]:
test_whole.shape,train_whole.shape,val_whole.shape

((1510, 768), (7045, 768), (1510, 768))

In [6]:
import numpy as np

# Indices where you want to insert empty arrays
empty_indices_train = [240, 1199, 1939, 2462, 2479, 3819, 4536, 4642, 6589, 7127, 7605]
empty_indices_test = [272, 662, 893, 1645, 1940]

# Insert empty arrays at specified indices for train embeddings
for index in empty_indices_train:
    empty_array = np.empty(shape=(1, 768), dtype=np.float64)
    rs_train_embeddings = np.insert(rs_train_embeddings, index, empty_array, axis=0)

# Insert empty arrays at specified indices for test embeddings
for index in empty_indices_test:
    empty_array = np.empty(shape=(1, 768), dtype=np.float64)
    rs_test_embeddings = np.insert(rs_test_embeddings, index, empty_array, axis=0)

# Verify the shapes after insertion
print(rs_train_embeddings.shape, rs_test_embeddings.shape)


(8052, 768) (2013, 768)


In [1]:
!pip install gdown

In [2]:
!gdown --id 1Fs0PTy_xPsoX5bg4QTCrX6JAaQCHRzis

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Fs0PTy_xPsoX5bg4QTCrX6JAaQCHRzis
To: /kaggle/working/train.pkl
100%|███████████████████████████████████████| 4.09M/4.09M [00:00<00:00, 206MB/s]


In [3]:
!gdown --id 1JLa-ELhUskQINi0syf3YJWBX8jcCXdn4

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1JLa-ELhUskQINi0syf3YJWBX8jcCXdn4
To: /kaggle/working/test.pkl
100%|███████████████████████████████████████| 1.02M/1.02M [00:00<00:00, 111MB/s]


In [17]:
!gdown -1qtKOid9F-WGMnk5SHywqGEC48o6VAh7A

Downloading...
From: https://drive.google.com/uc?id=1qtKOid9F-WGMnk5SHywqGEC48o6VAh7A
To: /kaggle/working/ls_train_embeddings.pkl
100%|██████████████████████████████████████| 49.5M/49.5M [00:01<00:00, 43.7MB/s]


In [18]:
!gdown 1j_aI3rr1W5wp5VmZRll6VHBXHPxprkAp

Downloading...
From: https://drive.google.com/uc?id=1j_aI3rr1W5wp5VmZRll6VHBXHPxprkAp
To: /kaggle/working/ls_test_embeddings.pkl
100%|██████████████████████████████████████| 12.4M/12.4M [00:00<00:00, 43.4MB/s]


In [22]:
!gdown --id 1n6IHWly66qSJfxrDSaIBcqm05jo-v6mt

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1n6IHWly66qSJfxrDSaIBcqm05jo-v6mt
To: /kaggle/working/rs_train_embeddings.pkl
100%|███████████████████████████████████████| 49.5M/49.5M [00:00<00:00, 137MB/s]


In [23]:
!gdown --id 1zToBjksCyT4DYAZ-0Oqj52nEXk0aqtdm

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1zToBjksCyT4DYAZ-0Oqj52nEXk0aqtdm
To: /kaggle/working/rs_test_embeddings.pkl
100%|██████████████████████████████████████| 12.4M/12.4M [00:00<00:00, 96.6MB/s]


In [29]:
from sklearn.linear_model import LogisticRegression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical

## manual tocknizer with root

In [30]:
BATCH_SIZE = 32
EPOCHS = 50
EMBED_SIZE = 768#86
LEARNING_RATE =  0.0053516485623658835

early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True)

In [56]:
X_train_gru_global = train_whole.reshape(train_whole.shape[0], 1, train_whole.shape[1]) # reshape input to allow for GRU
X_test_gru_global = test_whole.reshape(test_whole.shape[0], 1, test_whole.shape[1]) # reshape input to allow for GRU
X_val_gru_global = val_whole.reshape(val_whole.shape[0], 1,val_whole.shape[1]) # reshape input to allow for GRU


In [57]:
y_train=df_train['label']
y_test=df_test['label']
y_val=df_val['label']

In [58]:

# Create the GRU model
from tensorflow.keras.layers import GRU, Dense, Dropout

def build_gru():
    model = Sequential()
    model.add(tf.keras.layers.GRU(256,  input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]),return_sequences=False))
    model.add(Dropout(0.011691195516528197))
    model.add(Dense(8, activation='softmax'))

    model.compile(optimizer=tf.keras.optimizers.Adam(0.0053516485623658835),
                loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [68]:
model = build_gru()

history = model.fit(X_train_gru_global, to_categorical(y_train, num_classes=8), validation_data=(X_val_gru_global, to_categorical(y_val, num_classes=8)), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping_monitor])






Epoch 1/50
221/221 [==============================] - 4s 7ms/step - loss: 1.1310 - accuracy: 0.6001 - val_loss: 0.8557 - val_accuracy: 0.7026
Epoch 2/50
221/221 [==============================] - 1s 5ms/step - loss: 0.8170 - accuracy: 0.7192 - val_loss: 0.9120 - val_accuracy: 0.6815
Epoch 3/50
221/221 [==============================] - 1s 5ms/step - loss: 0.7605 - accuracy: 0.7255 - val_loss: 0.7952 - val_accuracy: 0.7245
Epoch 4/50
221/221 [==============================] - 1s 5ms/step - loss: 0.6817 - accuracy: 0.7524 - val_loss: 0.8145 - val_accuracy: 0.7126
Epoch 5/50
221/221 [==============================] - 1s 5ms/step - loss: 0.6451 - accuracy: 0.7682 - val_loss: 0.8286 - val_accuracy: 0.7278
Epoch 6/50
221/221 [==============================] - 1s 5ms/step - loss: 0.6275 - accuracy: 0.7760 - val_loss: 0.8573 - val_accuracy: 0.7060
Epoch 7/50
221/221 [==============================] - 1s 5ms/step - loss: 0.5898 - accuracy: 0.7895 - val_loss: 0.7888 - val_accuracy: 0.7417
Epoch 

In [69]:
predictions = model.predict(X_test_gru_global)
print(classification_report(y_test, predictions.argmax(axis=1)))

48/48 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.71      0.79      0.75       229
           1       0.71      0.77      0.74       200
           2       0.75      0.55      0.64       205
           3       0.65      0.59      0.62       185
           4       0.76      0.82      0.79       193
           5       0.77      0.94      0.85       156
           6       0.80      0.51      0.62       154
           7       0.77      0.91      0.83       188

    accuracy                           0.74      1510
   macro avg       0.74      0.73      0.73      1510
weighted avg       0.74      0.74      0.73      1510



In [67]:

# Create the GRU model
from tensorflow.keras.layers import GRU, Dense, Dropout

gru_classifier = Sequential()
gru_classifier.add(GRU(256, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=True))
gru_classifier.add(Dropout(0.3))

gru_classifier.add(GRU(128, return_sequences=True))
gru_classifier.add(Dropout(0.3))

gru_classifier.add(GRU(64))
gru_classifier.add(Dropout(0.3))

gru_classifier.add(Dense(8, activation='softmax'))

# Compile the model
gru_classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


## train and evalute Gru

gru_classifier.fit(X_train_gru_global,to_categorical(df_train['label'], num_classes=8) , validation_data=(X_val_gru_global, to_categorical(y_val, num_classes=8)), epochs=20,batch_size=128)


y_pred_gru = gru_classifier.predict(X_test_gru_global)

y_pred_classes = np.argmax(y_pred_gru, axis=1)

print("GRU",classification_report(df_test['label'], y_pred_classes))

Epoch 1/20
56/56 [==============================] - 8s 27ms/step - loss: 0.4224 - accuracy: 0.1371 - val_loss: 0.3755 - val_accuracy: 0.1490
Epoch 2/20
56/56 [==============================] - 1s 9ms/step - loss: 0.3837 - accuracy: 0.1476 - val_loss: 0.3588 - val_accuracy: 0.2821
Epoch 3/20
56/56 [==============================] - 1s 9ms/step - loss: 0.3434 - accuracy: 0.3242 - val_loss: 0.2968 - val_accuracy: 0.5318
Epoch 4/20
56/56 [==============================] - 1s 9ms/step - loss: 0.2891 - accuracy: 0.5001 - val_loss: 0.2483 - val_accuracy: 0.5940
Epoch 5/20
56/56 [==============================] - 0s 9ms/step - loss: 0.2479 - accuracy: 0.6034 - val_loss: 0.2232 - val_accuracy: 0.6252
Epoch 6/20
56/56 [==============================] - 0s 9ms/step - loss: 0.2259 - accuracy: 0.6419 - val_loss: 0.2027 - val_accuracy: 0.6636
Epoch 7/20
56/56 [==============================] - 0s 9ms/step - loss: 0.2083 - accuracy: 0.6805 - val_loss: 0.1945 - val_accuracy: 0.6947
Epoch 8/20
56/56 [=

In [63]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming you have already predicted y_pred_classes

# Print classification report
print("GRU", classification_report(df_test['label'], y_pred_classes))

# Create confusion matrix
conf_matrix = confusion_matrix(df_test['label'], y_pred_classes)

# Create a heatmap for the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt='g', xticklabels=[class_names], yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


GRU               precision    recall  f1-score   support

           0       0.68      0.87      0.77       229
           1       0.80      0.68      0.73       200
           2       0.74      0.55      0.63       205
           3       0.69      0.58      0.63       185
           4       0.81      0.78      0.79       193
           5       0.70      0.97      0.81       156
           6       0.72      0.56      0.63       154
           7       0.79      0.91      0.85       188

    accuracy                           0.74      1510
   macro avg       0.74      0.74      0.73      1510
weighted avg       0.74      0.74      0.73      1510



NameError: name 'class_names' is not defined

<Figure size 1000x800 with 0 Axes>

In [39]:
X_train_gru_global = ls_train_embeddings.reshape(ls_train_embeddings.shape[0], 1, ls_train_embeddings.shape[1]) # reshape input to allow for GRU

# Create the GRU model
from tensorflow.keras.layers import GRU, Dense, Dropout

gru_classifier = Sequential()
gru_classifier.add(GRU(256, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=True))
gru_classifier.add(Dropout(0.2))

gru_classifier.add(GRU(128, return_sequences=True))
gru_classifier.add(Dropout(0.2))

gru_classifier.add(GRU(64))
gru_classifier.add(Dropout(0.2))

gru_classifier.add(Dense(8, activation='softmax'))

# Compile the model
gru_classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


## train and evalute Gru

gru_classifier.fit(X_train_gru_global,to_categorical(df_train['label'], num_classes=8) , epochs=20,batch_size=128)

X_test_gru_global = ls_test_embeddings.reshape(ls_test_embeddings.shape[0], 1, ls_test_embeddings.shape[1])

y_pred_gru = gru_classifier.predict(X_test_gru_global)

y_pred_classes = np.argmax(y_pred_gru, axis=1)

print("GRU",classification_report(df_test['label'], y_pred_classes))

Epoch 1/20
63/63 [==============================] - 7s 8ms/step - loss: 0.3616 - accuracy: 0.3313
Epoch 2/20
63/63 [==============================] - 0s 7ms/step - loss: 0.2343 - accuracy: 0.6387
Epoch 3/20
63/63 [==============================] - 0s 7ms/step - loss: 0.1923 - accuracy: 0.7101
Epoch 4/20
63/63 [==============================] - 0s 8ms/step - loss: 0.1699 - accuracy: 0.7517
Epoch 5/20
63/63 [==============================] - 0s 7ms/step - loss: 0.1554 - accuracy: 0.7727
Epoch 6/20
63/63 [==============================] - 0s 7ms/step - loss: 0.1444 - accuracy: 0.7871
Epoch 7/20
63/63 [==============================] - 0s 8ms/step - loss: 0.1359 - accuracy: 0.8048
Epoch 8/20
63/63 [==============================] - 0s 7ms/step - loss: 0.1232 - accuracy: 0.8255
Epoch 9/20
63/63 [==============================] - 0s 8ms/step - loss: 0.1144 - accuracy: 0.8393
Epoch 10/20
63/63 [==============================] - 0s 7ms/step - loss: 0.1078 - accuracy: 0.8499
Epoch 11/20
63/63 [

## manuual tocknizer with light

In [ ]:
X_train_gru_global = ls_train_embeddings.reshape(ls_train_embeddings.shape[0], 1, ls_train_embeddings.shape[1]) # reshape input to allow for GRU

# Create the GRU model
from tensorflow.keras.layers import GRU, Dense, Dropout

gru_classifier = Sequential()
gru_classifier.add(GRU(256, input_shape=(X_train_gru_global.shape[1], X_train_gru_global.shape[2]), return_sequences=True))
gru_classifier.add(Dropout(0.2))

gru_classifier.add(GRU(128, return_sequences=True))
gru_classifier.add(Dropout(0.2))

gru_classifier.add(GRU(64))
gru_classifier.add(Dropout(0.2))

gru_classifier.add(Dense(8, activation='softmax'))

# Compile the model
gru_classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


## train and evalute Gru

gru_classifier.fit(X_train_gru_global,to_categorical(df_train['label'], num_classes=8) , epochs=20,batch_size=128)

X_test_gru_global = ls_test_embeddings.reshape(ls_test_embeddings.shape[0], 1, ls_test_embeddings.shape[1])

y_pred_gru = gru_classifier.predict(X_test_gru_global)

y_pred_classes = np.argmax(y_pred_gru, axis=1)

print("GRU",classification_report(df_test['label'], y_pred_classes))

# Saving data



In [110]:
import pickle



In [109]:

with open('train.pkl', 'wb') as f:
    pickle.dump(df_train, f)

with open('test.pkl', 'wb') as f:
    pickle.dump(df_test, f)

with open('val.pkl', 'wb') as f:
    pickle.dump(df_val, f)
